# Compute robustness maps for evaluated methods

## Read and preprocess data

In [ ]:
import random

from pathlib import Path
from typing import List, Tuple

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_csv("experiments/all_results.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

In [ ]:
ss_methods = df["selection_metric"].unique()
protocols = df["protocol"].unique()

# in some experiments mi_value was saved tih too big precision (differences 
# in 15th floating place) hence we need to round them 
df["mi_value"] = df["mi_value"].round(1)
mi_vals = df["mi_value"].unique()

# permute networks so that they're ordered as following:
# - real networks, ER SF
nets_ml = [
    'aucs', 'ckm_physicians', 'eu_transportation', 'lazega', 
    'er2', 'er3', 'er5', 'sf2', 'sf3', 'sf5',
]
nets_sl = ["eu_trans_1", "er1", "sf1"]
assert set(df["network"].unique()) == set(nets_ml + nets_sl)


#
#
_net_choice = "ml"  # <----------- SELECT network type to analyse
#
#

if _net_choice == "ml":
    nets = nets_ml
    net_case = "multilayer"
elif _net_choice == "sl":
    nets = nets_sl
    net_case = "singlelayer"
elif _net_choice == "all":
    nets = nets_ml + nets_sl
    net_case = "all" 
else:
    raise ValueError
print(f"Choice: {net_case}")

ss_methods_abbrv_map ={
    'degree_centrality': "deg-c",
    'greedy': "greedy",
    'k_shell': "k-sh",
    'k_shell_mln': "k-sh-m",
    'neighbourhood_size': "nghb-1s",
    'neighbourhood_2_hop_size': "nghb-2s",
    'page_rank': "p-rnk",
    'page_rank_mln': "p-rnk-m",
    'random': "random",
    'vote_rank': "v-rnk",
    'vote_rank_mln': "v-rnk-m",
}

nets_abbrv_map = {
    "aucs": "aucs",
    "ckm_physicians": "ckmp",
    "lazega": "lazega",
    "eu_transportation": "eutr-A",
    "eu_trans_1": "eutr-1",
    "er1": "er-1",
    "er2": "er-2",
    "er3": "er-3",
    "er5": "er-5",
    "sf1": "sf-1",
    "sf2": "sf-2",
    "sf3": "sf-3",
    "sf5": "sf-5",
}

In [ ]:
# for protocol OR we had budgets like [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30] 
# for protocol AND we had budgets like 
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
# but we'd like to skip wery low budgets where all methods are unaccurate and keep only:
# [15, 20, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40] 
rows_to_drop = df.loc[
    (df["protocol"] == "AND") & 
    (
        ~df["seeding_budget"].isin(
            {15, 20, 25, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40}
        )
    )
]
df = df.drop(rows_to_drop.index)
s_budgets = df["seeding_budget"].unique()

# drop unselected nets
rows_to_drop = df.loc[~df["network"].isin(set(nets))]
df = df.drop(rows_to_drop.index)


ss_methods, protocols, mi_vals, s_budgets, nets

## Convert data to a multidimensional matrices for each network type

In [ ]:
#
#
proto = "AND"  # <---------- change this to plot stats for AND!!!
#
#

s_budgets_proto = df.loc[df["protocol"] == proto]["seeding_budget"].unique()
cube = {}
consist_shape = []

for ssm in ss_methods:
    for net in nets:
        ddf = df.loc[
            (df["network"] == net) &
            (df["protocol"] == proto) &
            (df["selection_metric"] == ssm)
        ]
        ddf = pd.pivot_table(
            ddf, index="mi_value", columns="seeding_budget", values="gain"
        ).to_numpy()
        if len(consist_shape) == 0:
            consist_shape = ddf.shape
        if len(ddf) == 0:
            ddf = np.empty(shape=consist_shape)
            ddf[:] = np.nan

        if not cube.get(ssm):
            cube[ssm] = {}
        cube[ssm][net] = ddf

len(cube)

In [ ]:
list_arrays = []
ssm_axis = []
nets_axis = []

for ssm, net_dict in cube.items(): 
    ssm_axis.append(ssm)
    _arrs = []
    for idx, (net, arr) in enumerate(net_dict.items()):
        if len(nets_axis) == len(nets):
            assert nets_axis[idx] == net
        else:
            nets_axis.append(net)
        _arrs.append(arr)

    list_arrays.append(np.stack(_arrs))

cube_gain = np.stack(list_arrays)
nets_axis = np.array([nets_abbrv_map[nname] for nname in nets_axis])

# array with all gains concatenated for all test runs - check shape (L==R)
cube_gain.shape, (len(ss_methods), len(nets), len(mi_vals), len(s_budgets_proto))

In [ ]:
# we need to manually obtain indices of real, sf and er nets
print(nets_axis)

rn_names = nets_axis[rn_idx := [0, 1, 2, 3]]
er_names = nets_axis[en_idx := [4, 5, 6]]
sf_names = nets_axis[sn_idx := [7, 8, 9]]

print(rn_names)
print(er_names)
print(sf_names)

In [ ]:
gains_real_nets = cube_gain[:, rn_idx, ...]
gains_er_nets = cube_gain[:, en_idx, ...]
gains_sf_nets = cube_gain[:, sn_idx, ...]

In [ ]:
# take mean value across all network types
gains_real_mean = np.mean(gains_real_nets, 1)
gains_er_mean = np.mean(gains_er_nets, 1)
gains_sf_mean = np.mean(gains_sf_nets, 1)
gains_all_mean = np.mean(cube_gain, 1)

gains_real_nets = np.append(gains_real_nets, gains_real_mean[:, np.newaxis, ...], axis=1)
gains_er_nets = np.append(gains_er_nets, gains_er_mean[:, np.newaxis, ...], axis=1)
gains_sf_nets = np.append(gains_sf_nets, gains_sf_mean[:, np.newaxis, ...], axis=1)

rn_names = np.append(rn_names, "mean-real")
er_names = np.append(er_names, "mean-er")
sf_names = np.append(sf_names, "mean-sf")

# now we have in each array values of gain for each ssm, network, mi, sb enhanced by a slice ssm, mean_network, mi, sb

In [ ]:
# or take only averaged performance pre network type
gains_mean = np.array([gains_real_mean, gains_er_mean, gains_sf_mean, gains_all_mean]).transpose([1, 0, 2, 3])
avg_names = ["mean-real", "mean-er", "mean-sf", "mean-all"]

gains_mean.shape, avg_names

## Create robustness maps

In [ ]:
def grad(arr: np.ndarray, mi_ax: int, sb_ax: int) -> np.ndarray:
    arr_dmi = np.gradient(arr, axis=mi_ax)
    arr_dsb = np.gradient(arr, axis=sb_ax)
    arr_dmi_dsb = (np.sqrt((arr_dmi ** 2) + (arr_dsb ** 2)))[:, ::-1, :]
    return arr_dmi_dsb


def obtain_robust_idxs(arr: np.ndarray) -> List[Tuple[int, int]]:
    """Arr is a boolean matrix, output are coords."""
    robust_border_idxs = []
    for sb_idx, mi_tab in enumerate(arr.T):
        # print(mi_tab[::-1])
        for mi_idx, mi in enumerate(mi_tab[::-1]):
            if mi == True:
                robust_border_idxs.append([sb_idx, mi_idx])
                break
    return robust_border_idxs


def obtain_unrobust_idxs(arr: np.ndarray) -> List[Tuple[int, int]]:
    """Arr is a boolean matrix, output are coords."""
    unrobust_border_idxs = []
    for sb_idx, mi_tab in enumerate(arr.T):
        for mi_idx, mi in enumerate(mi_tab):
            if mi == True:
                unrobust_border_idxs.append([sb_idx, len(mi_tab) - 1 - mi_idx])
                break
    return unrobust_border_idxs


def convert_idxs(border_idxs: List[Tuple[int, int]]) -> np.ndarray:
    robust_border_coords = [
        [s_budgets_proto[sb_idx], mi_vals[mi_idx]] for 
        sb_idx, mi_idx in border_idxs
    ]
    robust_border_coords = np.array(robust_border_coords)
    return robust_border_coords


def obtain_robust_curve(arr: np.ndarray) -> np.ndarray:
    return convert_idxs(obtain_robust_idxs(arr))


def obtain_unrobust_curve(arr: np.ndarray) -> np.ndarray:
    return convert_idxs(obtain_unrobust_idxs(arr))


def log_func(x, a, b):
    return a * np.log(x) + b


def polynomial_3rd(x, a, b, c, d):
    return a * x ** 3 + b * x ** 2 + c * x + d


def sigmoid(x, a, b):
    return a * (np.exp(x) / (np.exp(x) + 1)) + b


def exp_func(x, a, b):
    return a * np.exp(x) + b


def optimize(func, curve_x, curve_y):
    parameters, covariance = curve_fit(func, curve_x, curve_y)
    std_err = np.sqrt(np.diag(covariance))
    fit_func = func(curve_x, *parameters)
    return (fit_func,
            {idx: val for idx, val in enumerate(std_err)},
            {idx: val for idx, val in enumerate(parameters)},
    )

In [ ]:
print(rn_names)
print(er_names)
print(sf_names)
print(avg_names)

In [ ]:
analysed_arr = gains_er_nets  # <---- select array to analyse
analysed_names = er_names  # <---- select proper names of nets
analysed_net = "mean-er"  # <---- select network to analyse

idx = int(np.where(np.array(analysed_names) == analysed_net)[0])

func = log_func  # <---- select function to aproximate curves with

In [ ]:
arr = analysed_arr[:, idx, ...]
print(arr.shape)

out_dir = Path("data/findings/robustness_maps")
out_dir.mkdir(exist_ok=True, parents=True)

vis_name = out_dir / f"map_proto-{proto}_net-{analysed_net}.png"
fit_err_name = out_dir / f"err_proto-{proto}_net-{analysed_net}.csv"
fit_par_name = out_dir / f"fit_proto-{proto}_net-{analysed_net}.csv"

In [ ]:
grad_arr = grad(arr, mi_ax=-2, sb_ax=-1)

curves = {}

for idx, ssm_grad in enumerate(grad_arr):

    method = ss_methods_abbrv_map[ss_methods[idx]]

    # sns.heatmap(ssm_grad, xticklabels=s_budgets_proto, yticklabels=mi_vals[::-1], cbar=False)
    # plt.title(f"raw gradinent for {method}")
    # plt.tight_layout()
    # plt.savefig(out_dir / f"raw_grad_{method}.png")
    # plt.close()

    ssm_grad = ssm_grad > 10

    # sns.heatmap(ssm_grad, xticklabels=s_budgets_proto, yticklabels=mi_vals[::-1], cbar=False)
    # plt.title(f"thresholded gradinent for {method}")
    # plt.tight_layout()
    # plt.savefig(out_dir / f"thre_grad_{method}.png")
    # plt.close()

    curves[method] = {
        "robust": obtain_robust_curve(ssm_grad),
        "unrobust": obtain_unrobust_curve(ssm_grad)
    }

In [ ]:
# a particular visualisation on curve fitting

# method  = "v-rnk-m"

# grad_robust = curves[method]["robust"]
# grad_unrobust = curves[method]["unrobust"]

# pts = np.linspace(1, max(s_budgets_proto))  

# fit_func_robust, std_err_robust, par_robust = optimize(log_func, grad_robust[:, 0], grad_robust[:, 1])
# fit_func_robust = log_func(pts, *par_robust.values())

# fit_func_unrobust, std_err_unrobust, par_unrobust = optimize(log_func, grad_unrobust[:, 0], grad_unrobust[:, 1])
# fit_func_unrobust = log_func(pts, *par_unrobust.values()) 

# plt.ylim(min(mi_vals), max(mi_vals))
# plt.xlim(min(pts), max(pts))

# plt.fill_between(pts, fit_func_robust, alpha=.5, color="green")
# plt.fill_between(pts, fit_func_unrobust, max(mi_vals) * len(fit_func_unrobust), alpha=.5, color="red")
# plt.fill_between(pts, fit_func_unrobust, fit_func_robust, alpha=.5, color="orange")

# plt.title(f"map for {method}")
# plt.tight_layout()
# plt.savefig(out_dir / f"map_{method}.png")
# plt.close()

# print("Robust coeff: ", par_robust, "std_err:", std_err_robust)
# print("Unobust coeff: ", par_unrobust, "std_err:", std_err_unrobust)

In [ ]:
color_iter = iter(plt.cm.jet(np.linspace(0, 1, len(curves))))
line_width = 2

fit_errors = {}
fit_params = {}

fig, ax = plt.subplots(nrows=1, ncols=1)

for idx, (ssm, curve) in enumerate(curves.items()):
    
    pts_robust = curve["robust"]
    pts_unrobust = curve["unrobust"]

    try:
        fit_func_robust, std_err_robust, par_robust = optimize(func, pts_robust[:, 0], pts_robust[:, 1])
        fit_func_unrobust, std_err_unrobust, par_unrobust = optimize(func, pts_unrobust[:, 0], pts_unrobust[:, 1])

        # if fitted function is too short - expand it to be plotted on entire canvas
        if len(fit_func_robust) < len(s_budgets):
            fit_func_robust = func(s_budgets, *par_robust.values())
        if len(fit_func_unrobust) < len(s_budgets):
            fit_func_unrobust = func(s_budgets, *par_unrobust.values()) 
        pts = s_budgets           

    except BaseException as e:
        if ssm != "greedy":
            raise e
        continue

    fit_errors[(ssm, "robust")] = std_err_robust
    fit_errors[(ssm, "unrobust")] = std_err_unrobust
    fit_params[(ssm, "robust")] = par_robust
    fit_params[(ssm, "unrobust")] = par_unrobust

    color = next(color_iter)
    alpha = .8
    linestyle=(4 * random.random(), (2, 2))

    if ssm == "random":
        ax.fill_between(
            pts, fit_func_robust, alpha=.5, color="green", 
        )
    elif ssm == "greedy":
        try:
            ax.fill_between(
                pts,
                fit_func_unrobust,
                max(mi_vals) * len(fit_func_unrobust),
                alpha=.5,
                color="red",
            )
        except BaseException as e:
            pass
    else:
        plt.plot(
            pts,
            fit_func_robust,
            label=ssm,
            marker='',
            color=color,
            linewidth=line_width,
            linestyle=linestyle,
            alpha=alpha,
        )

ax.set_xlim(min(s_budgets_proto), max(s_budgets_proto))
# ax.set_xscale("log")
ax.set_xticks(s_budgets_proto)
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.set_ylim(min(mi_vals), max(mi_vals))
ax.legend(loc="upper left")
ax.grid()
# fig.suptitle(vis_name.stem)
fig.set_size_inches(6, 5)
fig.tight_layout()
fig.savefig(vis_name, dpi=300)

In [ ]:
pd.DataFrame(fit_errors).T.to_csv(fit_err_name)
pd.DataFrame(fit_params).T.to_csv(fit_par_name)

## Compute avg error of curve fitting

In [ ]:
errs = {}

for err in out_dir.glob("[err]*.csv"):
    print(err)
    raw_err_df = pd.read_csv(
        err,
        names=[
            "ssm",
            "curve",
            "parameter_0",
            "parameter_1",
            # "parameter_2",
            # "parameter_3"
        ],
        header=1
    )
    unrobust_to_drop = raw_err_df.loc[(raw_err_df["curve"] == "unrobust") & (raw_err_df["ssm"] != "greedy")].index
    robust_to_drop = raw_err_df.loc[(raw_err_df["curve"] == "robust") & (raw_err_df["ssm"] == "greedy")].index
    err_df = raw_err_df.drop(unrobust_to_drop, axis=0).drop(robust_to_drop, axis=0)

    errs[err.stem[4:]] = err_df

In [ ]:
pars = {}

for par in out_dir.glob("[fit]*.csv"):
    print(par)
    raw_par_df = pd.read_csv(
        par,
        names=[
            "ssm",
            "curve",
            "parameter_0",
            "parameter_1",
            # "parameter_2",
            # "parameter_3"
        ],
        header=1
    )
    
    unrobust_to_drop = raw_par_df.loc[(raw_par_df["curve"] == "unrobust") & (raw_par_df["ssm"] != "greedy")].index
    robust_to_drop = raw_par_df.loc[(raw_par_df["curve"] == "robust") & (raw_par_df["ssm"] == "greedy")].index
    par_df = raw_par_df.drop(unrobust_to_drop, axis=0).drop(robust_to_drop, axis=0)

    pars[par.stem[4:]] = par_df


In [ ]:
mean_errs_prct = {}

for figure_name, par_df in pars.items():
    err_df = errs[figure_name]

    _ = pd.merge(err_df, par_df, on=["ssm", "curve"], suffixes=["_err", "_par"])
    _["parameter_0_prct_err"] = np.abs(100 * _["parameter_0_err"] / _["parameter_0_par"])
    _["parameter_1_prct_err"] = np.abs(100 * _["parameter_1_err"] / _["parameter_1_par"])
    # _["parameter_2_prct_err"] = np.abs(100 * _["parameter_2_err"] / _["parameter_2_par"])
    # _["parameter_3_prct_err"] = np.abs(100 * _["parameter_3_err"] / _["parameter_3_par"])

    mean_errs_prct[figure_name] = {
        "parameter_0_prct_err": _["parameter_0_prct_err"].mean(),
        "parameter_1_prct_err": _["parameter_1_prct_err"].mean(),
        # "parameter_2_prct_err": _["parameter_2_prct_err"].mean(),
        # "parameter_3_prct_err": _["parameter_3_prct_err"].mean(),
    }
    

In [ ]:
mean_errs_prct_df = pd.DataFrame(mean_errs_prct).T
mean_errs_prct_df

In [ ]:
mean_errs_prct_df.to_csv(out_dir / "mean_errors.csv")